In [1]:
%pip install langchain -qq
%pip install langchain_community -qq
%pip install langchain-google-genai -qq
%pip install python-dotenv -qq
%pip install streamlit -qq
%pip install langchain_experimental -qq
%pip install sentence-transformers -qq
%pip install langchain_chroma -qq
%pip install langchainhub -qq
%pip install pypdf -qq
%pip install rapidocr-onnxruntime -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 808.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.6/396.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.8/289.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import os
from google.colab import userdata


os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
g_api_key = os.getenv("GOOGLE_API_KEY")

Integrating RAG with Chatbot (full application code)

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Mind's Eye of LLMs-VoT Paper.pdf to Mind's Eye of LLMs-VoT Paper.pdf


In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Mind's Eye of LLMs-VoT Paper.pdf")
data = loader.load()  # entire PDF is loaded as a single Document
data

[Document(metadata={'source': "Mind's Eye of LLMs-VoT Paper.pdf", 'page': 0}, page_content='Mind’s Eye of LLMs: Visualization-of-Thought Elicits\nSpatial Reasoning in Large Language Models\nWenshan Wu Shaoguang Mao Yadong Zhang Yan Xia Li Dong Lei Cui Furu Wei\nMicrosoft Research\nhttps://aka.ms/GeneralAI\nAbstract\nLarge language models (LLMs) have exhibited impressive performance in language\ncomprehension and various reasoning tasks. However, their abilities in spatial\nreasoning, a crucial aspect of human cognition, remain relatively unexplored.\nHuman possess a remarkable ability to create mental images of unseen objects and\nactions through a process known as the Mind’s Eye , enabling the imagination of\nthe unseen world. Inspired by this cognitive capacity, we propose Visualization-\nof-Thought ( VoT) prompting. V oT aims to elicit spatial reasoning of LLMs by\nvisualizing their reasoning traces, thereby guiding subsequent reasoning steps. We\nemployed V oT for multi-hop spatial

In [9]:
l = len(data)
l

33

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  104


In [11]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()



embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734449505806,
 0.01813092641532421]

In [12]:
vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [14]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})


In [21]:
import os
import google.generativeai as genai


model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])

while True:
  prompt = input("Hey Human! How can I assist you?: ")
  if (prompt.lower() == "exit"):
    break

  # Retrieve relevant documents
  retrieved_docs = retriever.invoke(prompt)

  # Generate response with context from retrieved documents
  context = [doc.page_content for doc in retrieved_docs[:5]]  # Use top 5 retrieved documents
  response = chat.send_message(prompt, stream=True)
  for chunk in response:
    if chunk.text:
      print(chunk.text)

Hey Human! How can I assist you?: what is mind's eye of LLM mentioned in the paper?
The term "mind's eye" of LLM is not mentioned in the
 provided paper.
Hey Human! How can I assist you?: what is spacial reasoning in LLM?
**Spatial reasoning in LLMs** refers to the ability of large language models (
LLMs) to understand and reason about spatial relationships and configurations. This includes tasks such as:

* **Spatial orientation:** Understanding the relative positions and orientations of
 objects in space.
* **Spatial transformation:** Mentally rotating, translating, or otherwise transforming objects in space.
* **Spatial visualization:** Creating mental representations of objects and their spatial relationships.
* **Spatial planning:** Reasoning about the spatial consequences of actions or events.

LLMs are trained on massive datasets of
 text and code, which includes a variety of spatial information. This allows them to learn to represent and reason about spatial concepts. For examp